Code to accompany Machine Learning Recipes #8. We'll write a Decision Tree Classifier, in pure Python. Below each of the methods, I've written a little demo to help explain what it does.

In [1]:
# For Python 2/3 compatability
from __future__ import print_function
from math import log
import pandas as pd
import numpy as np
import random

In [2]:
# data reading
my_file = open('postoperative.txt','r')
lines = my_file.readlines()

data = [[] for i in range(len(lines))]


for i in range(len(lines)):
    lines[i]=lines[i][:-1]
    lines[i]=lines[i].split(',')

    if lines[i][7]=='?':
        lines[i][7]=10
    if lines[i][8]=="I":
        continue
    if lines[i][8]=="A ":
        lines[i][8]='A'

    lines[i][7]=int(lines[i][7])
    data[i].append(lines[i])
    data[i]=sum(data[i], [])
    
data[0][0]='mid'
data[-1][-1]='S'
data=[x for x in data if x != []]

## split training and testing
randtran=random.sample(range(88),70)
testing_data=[data[i] for i in range(88) if i not in randtran]
training_data=[data[num] for num in randtran]

print(len(training_data))
print(len(testing_data))
training_data[0]

70
18


['mid', 'mid', 'good', 'mid', 'unstable', 'stable', 'unstable', 10, 'S']

In [3]:
# Toy dataset.
# Format: each row is an example.
# The last column is the label.
# The first two columns are features.
# Feel free to play with it by adding more features & examples.
# Interesting note: I've written this so the 2nd and 5th examples
# have the same features, but different labels - so we can see how the
# tree handles this case.
#training_data = [
#    ['Green', 3, 'Apple'],
#    ['Yellow', 3, 'Apple'],
#    ['Red', 1, 'Grape'],
#    ['Red', 1, 'Grape'],
#    ['Yellow', 3, 'Grape'],
#]

In [4]:
# Column labels.
# These are used only to print the tree.
header = ["L-CORE", "L-SURF","L-O2","L-BP","SURF-STBL","CORE-STBL", "BP-STBL",'COMFORT','Decision']

In [5]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [6]:
#######
# Demo:
unique_vals(training_data,0)
# unique_vals(training_data, 1)
#######

{'high', 'low', 'mid'}

In [7]:
uni_label=list(unique_vals(training_data,-1))
uni_label

['S', 'A']

In [8]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [9]:
#######
# Demo:
class_counts(training_data)
list(class_counts(training_data).items())
#######

[('S', 17), ('A', 53)]

In [10]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [11]:
#######
# Demo:
is_numeric(training_data[0][7])
# is_numeric("Red")
#######

True

In [12]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        return val >= self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
            condition = ">="
            return "Is %s %s %s?" % (
                header[self.column], condition, str(self.value))

In [13]:
class Questionnom:
    def __init__(self, column):
        self.column = column
    def __repr__(self):
        return "What's the %s of instances?" % (header[self.column])

In [14]:
#######
# Demo:
# Let's write a question for a numeric attribute
Question(7, 10)

Is COMFORT >= 10?

In [15]:
Questionnom(0)

What's the L-CORE of instances?

In [16]:
def partition(rows, question):
    """Partitions a dataset for numerical attributes.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [17]:
#######
# Demo:
# Let's partition the training data based on whether rows are Red.
true_rows, false_rows = partition(training_data, Question(7, 11))
# This will contain all the 'Red' rows.
true_rows[:3]

[['low', 'high', 'good', 'high', 'unstable', 'stable', 'mod-stable', 15, 'A'],
 ['high',
  'high',
  'excellent',
  'high',
  'unstable',
  'stable',
  'unstable',
  15,
  'A'],
 ['mid', 'high', 'good', 'mid', 'unstable', 'stable', 'unstable', 15, 'A']]

In [18]:
# This will contain everything else.
false_rows[:3]
#######

[['mid', 'mid', 'good', 'mid', 'unstable', 'stable', 'unstable', 10, 'S'],
 ['low', 'low', 'good', 'mid', 'unstable', 'stable', 'unstable', 10, 'S'],
 ['low', 'mid', 'good', 'mid', 'unstable', 'stable', 'stable', 10, 'A']]

In [19]:
def partitionnom(rows,column,data):
    """Partitions a dataset for nominal attributes.
    """
    unique=list(unique_vals(data,column))
    headers = [[] for i in range(len(unique))]
    
    for i in range(len(unique)):
        for row in rows:
            if row[column]==unique[i]:
                headers[i].append(row)
    return headers       

In [21]:
partitionnom(testing_data,6,data)[0][:3]

[['high',
  'high',
  'excellent',
  'high',
  'unstable',
  'stable',
  'unstable',
  10,
  'A'],
 ['low', 'low', 'good', 'mid', 'unstable', 'stable', 'unstable', 10, 'A'],
 ['mid', 'mid', 'good', 'mid', 'stable', 'stable', 'unstable', 10, 'A']]

In [22]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [23]:
#self add
def entropy(rows):
    """Calculate the Gain by Entropy for a list of rows.
    """
    counts = class_counts(rows)
    impurity = 0
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl*log(prob_of_lbl,2)
    return impurity

In [24]:
#######
# Demo:
# Let's look at some example to understand how Gini Impurity works.
#
# First, we'll look at a dataset with no mixing.
no_mixing = [['Apple'],
              ['Apple']]
class_counts(no_mixing)

# this will return 0
print(gini(no_mixing))

#selfadd
print(entropy(no_mixing))

0.0
0.0


In [24]:
# Now, we'll look at a dataset with many different labels
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
print(gini(lots_of_mixing))
#######

#selfadd
print(entropy(lots_of_mixing))

0.7999999999999998
2.321928094887362


In [26]:
def count_list(l):
    count = 0
    for e in l:
        if isinstance(e, list):
            count = count + 1 + count_list(e)
    return count

In [29]:
count=count_list(partitionnom(training_data,0,data))-int(len(partitionnom(training_data,0,data)))
count

70

In [25]:
def info_gain(left, right, current_uncertainty):
    """Information Gain for numerical attrubutes.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [30]:
def info_gainnom(header,current_uncertainty):
    """Information Gain for nominal attrubutes.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    gainnow=0
    for lst in header:
        p=float(len(lst))/(count_list(header)-len(header))
        gainnow+=p*gini(lst)
    return current_uncertainty-gainnow

In [31]:
#selfadd
def info_gainratio(left, right, current_uncertainty):
    """Information Gain Ratio for numerical attributes.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes, and split by splitinfo.
    """
    p = float(len(left)) / (len(left) + len(right))
    gains = current_uncertainty - p * entropy(left)- (1 - p) *entropy(right)
    return gains/(-p*log(p,2)-(1-p)*log((1-p),2))

In [32]:
#selfadd
def info_gainrationom(header,current_uncertainty):
    """Information Gain Ratio for nominal attrubutes.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    gainnow=0
    split=0
    
    for lst in header:
        p=float(len(lst))/(count_list(header)-len(header))
        gainnow+=p*entropy(lst)
        split-=p*log(p,2)
    return (current_uncertainty-gainnow)/split

In [33]:
#self add
def auc(left, right,uni_label):
    """Calculate the auc for different partitions for numerical attributes.Doesn't need current uncertainty because doesn't depend
    on parent node but only child nodes
    """
    pos=uni_label[1]
    neg=uni_label[0]

    left=class_counts(left)
    right=class_counts(right)
    if pos not in left:
        left[pos]=0
    if neg not in left:
        left[neg]=0
    if pos not in right:
        right[pos]=0
    if neg not in right:
        right[neg]=0

    if left[pos]> right[pos] :
        tp=left[pos]/(right[pos]+left[pos])
        fp=left[neg]/(right[neg]+left[neg])
    else:
        tp=right[pos]/(right[pos]+left[pos])
        fp=right[neg]/(right[neg]+left[neg])
    return float(tp-fp+1)/2

In [34]:
#self add
def aucnom(header,uni_label):
    """Calculate the auc for different partitions for nominal attributes.Doesn't need current uncertainty because doesn't depend
    on parent node but only child nodes
    """
    pos=uni_label[1]
    neg=uni_label[0]
    posnum=0
    negnum=0
    auc=0
    head=[]
       
    for ls in header: 
        v=class_counts(ls)
        if pos not in v:v[pos]=0
        if neg not in v:v[neg]=0
        head.append(list(v.items()))

    for i in range(len(header)):
        v=class_counts(header[i])
        if pos not in v:v[pos]=0
        if neg not in v:v[neg]=0
        val=v[pos]
        posnum+=val
        
        for j in range(len(header)-1,i,-1):            
            vcomp=class_counts(header[j])
            if pos not in vcomp:vcomp[pos]=0
            if neg not in vcomp:vcomp[neg]=0            
            valcomp=vcomp[pos]
            
            if valcomp>val:
                head[i]=list(vcomp.items())
                head[j]=list(v.items())
                
    negnum=count_list(header)-len(header)-posnum

    for ls in head:ls.sort()
    if head[0][0][0]==pos: 
        num=0
        notnum=1
    else: 
        num=1
        notnum=0
        
    for i in range(1,len(head)):
        val=head[i][notnum][1]
        val1=head[i][num][1]
        valnext=0
        for j in range(i):
            add=head[j][num][1]
            valnext+=add     
        auc+=(val)*(2*valnext)+val1
        
    return float(auc)/(2*posnum*negnum)

In [35]:
#######
# Demo:
# Calculate the uncertainy of our training data.
current_uncertainty = gini(training_data)
print(current_uncertainty)

#selfadd
current_uncertainty2= entropy(training_data)
current_uncertainty2

0.39551020408163273


0.8435070855739036

In [36]:
print(info_gainnom(partitionnom(training_data,0,data),current_uncertainty))
print(info_gainrationom(partitionnom(training_data,0,data),current_uncertainty2))
aucnom(partitionnom(training_data,0,data),uni_label)

0.0024115153307699977
0.0034457845905465855


0.27502579979360164

In [37]:
# How much information do we gain by partioning on 'Green'?
true_rows, false_rows = partition(training_data, Question(7, 11))


print(info_gain(true_rows, false_rows, current_uncertainty))
#selfadd
print(info_gainratio(true_rows, false_rows, current_uncertainty2))
auc(true_rows,false_rows,uni_label)

0.006356764928193592
0.014294237561698057


0.5598555211558307

In [38]:
# What about if we partioned on 'Red' instead?
true_rows, false_rows = partition(training_data, Question(7,7))

print(info_gain(true_rows, false_rows, current_uncertainty))
#selfadd
print(info_gainratio(true_rows, false_rows, current_uncertainty2))
print(auc(true_rows,false_rows,uni_label))

0.002135462880804573
0.06093145293200665
0.49019607843137253


In [40]:
true_rows, false_rows = partition(training_data, Question(7,13))
true_rows[:2]

[['low', 'high', 'good', 'high', 'unstable', 'stable', 'mod-stable', 15, 'A'],
 ['high',
  'high',
  'excellent',
  'high',
  'unstable',
  'stable',
  'unstable',
  15,
  'A']]

In [42]:
false_rows[:2]

[['mid', 'mid', 'good', 'mid', 'unstable', 'stable', 'unstable', 10, 'S'],
 ['low', 'low', 'good', 'mid', 'unstable', 'stable', 'unstable', 10, 'S']]

In [49]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions=class_counts(rows)

In [50]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch=true_branch
        self.false_branch = false_branch

In [51]:
class Decision_Nodenom:
    """A Decision Node asks a question.

    This holds a reference to the question, and to several child nodes.
    """
    def __init__(self,
                 question,
                 cutpoint,
                 all_branch):
        self.question = question
        self.cutpoint= cutpoint
        self.all_branch = all_branch

In [52]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    elif isinstance(node,Decision_Node):
        print (spacing + str(node.question))

        # Call this function recursively on the true branch
        print (spacing + '--> True:')
        print_tree(node.true_branch, spacing + "  ")

        # Call this function recursively on the false branch
        print (spacing + '--> False:')
        print_tree(node.false_branch, spacing + "  ")
    
    else:
        print(spacing + str(node.question))
        for i in range(len(node.all_branch)):
            print(spacing+'-->'+str(node.cutpoint[i]))
            print_tree(node.all_branch[i],spacing + "  ")

In [53]:
def find_best_split(rows,data):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain.
    data usage : to find all possible levels of a column
    """
    
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    num=None
    column=None
    cutpoint=None
    branch=None
    restart=None
    
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    #stopping criteria 1: all instances in node contain same class value
    sameclass=set([row[-1] for row in rows])
    if len(sameclass)==1:
        num,branch,column,cutpoint,best_gain, best_question = num,branch,column,cutpoint,best_gain,best_question
        return num,branch,column,cutpoint,best_gain, best_question
    
    for col in range(n_features):  # for each feature
        values = set([row[col] for row in data])  # unique values in the column
        values=list(values)
        
        # numerical variables!----------------------------------------------------
        if is_numeric(values[0]):
            for val in values:  # for each value deciding cutpoint

                question = Question(col, val)

                # try splitting the dataset
                true_rows, false_rows = partition(rows, question)
                
                #stopping criteria 2: min number of instances in leaf
                if len(true_rows) < 3 or len(false_rows) < 3:
                    continue
                    
                #stopping criteria 3: all instances in node contain same attributes value
                # Skip this split if it doesn't divide the dataset.                   
                if len(true_rows) == 0 or len(false_rows) == 0:
                    continue

                # Calculate the information gain from this split
                gain = info_gain(true_rows, false_rows, current_uncertainty) 
                if gain > best_gain:
                    num,branch,column,cutpoint,best_gain, best_question = True,2,col,val,gain, question

        #nominal variables--------------------------------------------------------
        else:
            question=Questionnom(col)
                
            #stopping criteria 2: min number of instances in leaf
            for ls in partitionnom(rows,col,data):
                if len(ls) <3:
                    restart=True
                    break
            if restart:
                continue
                
            #stopping criteria 3: all instances in node contain same attributes value
            #Skip this split if it doesn't divide the dataset.                
            if len(partitionnom(rows,col,data)) == 1:
                continue
                    
            # Calculate the information gain from this split
            gain = info_gainnom(partitionnom(rows,col,data), current_uncertainty)
            if gain > best_gain:
                 num,branch,column,cutpoint,best_gain,best_question = False,len(partitionnom(rows,col,data)),col,values,gain,question
        #--------------------------------------------------------------------------
            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
    #num:numerical or not, branch: number of branches, column: col. chose this time, cutpoint:all possible levels of that column
    return num,branch,column,cutpoint,best_gain, best_question 
 

In [61]:
# Demo:Find the best question to ask first for ourdataset.
num,branch,col,cutpoint,best_gain, best_question = find_best_split(training_data,data)
print(num,branch,col,cutpoint,best_gain,best_question)

num,branch,col,cutpoint,best_gain, best_question = find_best_splitratio(training_data,data)
print(num,branch,col,cutpoint,best_gain,best_question)

num,branch,col,cutpoint,best_gain, best_question = find_best_splitauc(training_data,data)
print(num,branch,col,cutpoint,best_gain,best_question)

True 2 7 15 0.006356764928193592 Is COMFORT >= 15?
True 2 7 15 0.014294237561698057 Is COMFORT >= 15?
True 2 7 15 0.5598555211558307 Is COMFORT >= 15?


In [62]:
def build_treegini(rows,data):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    num,branch,column,cutpoint,gain,question = find_best_split(rows,data)
    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    if num==True:
        true_rows, false_rows = partition(rows, question)
        # Recursively build the true branch.
        true_branch = build_treegini(true_rows,data)

        # Recursively build the false branch.
        false_branch = build_treegini(false_rows,data)
        return Decision_Node(question,true_branch, false_branch)
    
    elif num==False:
        store=[0]*branch
        for i in range(branch):
            store[i]=build_treegini(partitionnom(rows,column,data)[i],data)   
        return Decision_Nodenom(question,cutpoint,store)
        
    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # depending on the answer.


In [63]:
my_treegini = build_treegini(training_data,data)
print_tree(my_treegini)

Is COMFORT >= 15?
--> True:
  What's the L-SURF of instances?
  -->low
    Predict {'S': 2, 'A': 1}
  -->mid
    Predict {'A': 5, 'S': 3}
  -->high
    Predict {'A': 4, 'S': 1}
--> False:
  What's the L-CORE of instances?
  -->low
    Predict {'A': 8, 'S': 4}
  -->mid
    Predict {'S': 9, 'A': 27}
  -->high
    Predict {'A': 6}


In [64]:
def find_best_splitratio(rows,data):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    num=None
    column=None
    cutpoint=None
    branch=None
    restart=None
    
    current_uncertainty = entropy(rows)
    n_features = len(rows[0]) - 1  # number of columns

    #stopping criteria 1: all instances in node contain same class value
    sameclass=set([row[-1] for row in rows])
    
    if len(sameclass)==1:
        num,branch,column,cutpoint,best_gain, best_question = num,branch,column,cutpoint,best_gain,best_question
        return num,branch,column,cutpoint,best_gain, best_question
    
    for col in range(n_features):  # for each feature
                   
        values = set([row[col] for row in data])  # unique values in the column
        values=list(values)
        
        # numerical variables!----------------------------------------------------
        if is_numeric(values[0]):
            for val in values:  # for each value deciding cutpoint

                question = Question(col, val)

                # try splitting the dataset
                true_rows, false_rows = partition(rows, question)
                
                #stopping criteria 2: number of instances in node is less than a threshold        
                if len(true_rows) < 3 or len(false_rows) < 3:
                    continue
                
                #stopping criteria 3: all instances in node contain same attributes value
                # Skip this split if it doesn't divide the dataset.
                if len(true_rows) == 0 or len(false_rows) == 0:
                    continue

                # Calculate the information gainratio from this split
                gain = info_gainratio(true_rows, false_rows, current_uncertainty) 
                if gain > best_gain:
                    num,branch,column,cutpoint,best_gain, best_question = True,2,col,val,gain, question

        #nominal variables--------------------------------------------------------
        else:
            question=Questionnom(col)
                
            #stopping criteria 2: number of instances in node is less than a threshold
            for ls in partitionnom(rows,col,data):
                if len(ls) <3:
                    restart=True
                    break
            if restart:
                continue
                
            #stopping criteria 3: all instances in node contain same attributes value
            # Skip this split if it doesn't divide the dataset.                
            if len(partitionnom(rows,col,data)) ==1:
                    continue
                    
            # Calculate the information gainratio from this split
            gain = info_gainrationom(partitionnom(rows,col,data), current_uncertainty)
            if gain > best_gain:
                 num,branch,column,cutpoint,best_gain,best_question = False,len(partitionnom(rows,col,data)),col,values,gain,question
        #--------------------------------------------------------------------------
            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.

    return num,branch,column,cutpoint,best_gain, best_question 
 

In [65]:
def build_treeratio(rows,data):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    num,branch,column,cutpoint,gain,question = find_best_splitratio(rows,data)
    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    elif num==True:
        true_rows, false_rows = partition(rows, question)

        # Recursively build the true branch.
        true_branch = build_treeratio(true_rows,data)

        # Recursively build the false branch.
        false_branch = build_treeratio(false_rows,data)
        return Decision_Node(question,true_branch, false_branch)
    
    elif num==False:
        store=[0]*branch
        for i in range(branch):
            store[i]=build_treeratio(partitionnom(rows,column,data)[i],data)   
        return Decision_Nodenom(question,cutpoint,store)
        
    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # depending on the answer.


In [66]:
my_treeratio = build_treeratio(training_data,data)
print_tree(my_treeratio)

Is COMFORT >= 15?
--> True:
  What's the L-SURF of instances?
  -->low
    Predict {'S': 2, 'A': 1}
  -->mid
    Predict {'A': 5, 'S': 3}
  -->high
    Predict {'A': 4, 'S': 1}
--> False:
  What's the L-CORE of instances?
  -->low
    Predict {'A': 8, 'S': 4}
  -->mid
    Predict {'S': 9, 'A': 27}
  -->high
    Predict {'A': 6}


In [67]:
def find_best_splitauc(rows,data):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    num=None
    column=None
    cutpoint=None
    branch=None
    restart=None

    n_features = len(rows[0]) - 1  # number of columns

    #stopping criteria 1: all instances in node contain same class value
    sameclass=set([row[-1] for row in rows])
    if len(sameclass)==1:
        num,branch,column,cutpoint,best_gain, best_question = num,branch,column,cutpoint,best_gain,best_question
        return num,branch,column,cutpoint,best_gain, best_question
    
    
    for col in range(n_features):  # for each feature
                   
        values = set([row[col] for row in data])  # unique values in the column
        values=list(values)
        
        # numerical variables!----------------------------------------------------
        if is_numeric(values[0]):
            for val in values:  # for each value deciding cutpoint

                question = Question(col, val)

                # try splitting the dataset
                true_rows, false_rows = partition(rows, question)
                
                #stopping criteria 2: number of instances in node is less than a threshold        
                if len(true_rows) < 3 or len(false_rows) < 3:
                    continue
                
                #stopping criteria 3: all instances in node contain same attributes value
                # Skip this split if it doesn't divide the dataset.
                if len(true_rows) == 0 or len(false_rows) == 0:
                    continue

                # Calculate the information gain from this split
                gain = auc(true_rows, false_rows,uni_label) 
                if gain > best_gain:
                    num,branch,column,cutpoint,best_gain, best_question = True,2,col,val,gain, question

        #nominal variables--------------------------------------------------------
        else:
            question=Questionnom(col)
                
            #stopping criteria 2: number of instances in node is less than a threshold
            for ls in partitionnom(rows,col,data):
                if len(ls) <3:
                    restart=True
                    break
            if restart:
                continue
                
            #stopping criteria 3: all instances in node contain same attributes value
            # Skip this split if it doesn't divide the dataset.                
            if len(partitionnom(rows,col,data)) ==1:
                    continue
                    
            # Calculate the information gain from this split
            gain = aucnom(partitionnom(rows,col,data),uni_label)
            if gain > best_gain:
                 num,branch,column,cutpoint,best_gain,best_question = False,len(partitionnom(rows,col,data)),col,values,gain,question
        #--------------------------------------------------------------------------
            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.

    return num,branch,column,cutpoint,best_gain, best_question 
 

In [68]:
def build_treeauc(rows,data):
    num,branch,column,cutpoint,gain,question = find_best_splitauc(rows,data)
    # Base case: no further gain(3 stopping criterions met,or gain is truly 0)
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    if num==True:
        true_rows, false_rows = partition(rows, question)

        # Recursively build the true branch.
        true_branch = build_treeauc(true_rows,data)

        # Recursively build the false branch.
        false_branch = build_treeauc(false_rows,data)
        
        return Decision_Node(question,true_branch, false_branch)
    
    elif num==False:
        store=[0]*branch
        for i in range(branch):
            store[i]=build_treeauc(partitionnom(rows,column,data)[i],data)   
        return Decision_Nodenom(question,cutpoint,store)
        
    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # depending on the answer


In [69]:
my_treeauc = build_treeauc(training_data,data)
print_tree(my_treeauc)

Is COMFORT >= 15?
--> True:
  What's the L-CORE of instances?
  -->low
    Predict {'A': 2, 'S': 1}
  -->mid
    Predict {'A': 7, 'S': 3}
  -->high
    Predict {'A': 1, 'S': 2}
--> False:
  What's the L-O2 of instances?
  -->excellent
    What's the L-SURF of instances?
    -->low
      Predict {'A': 5, 'S': 1}
    -->mid
      Predict {'A': 13, 'S': 3}
    -->high
      Predict {'A': 3, 'S': 1}
  -->good
    Predict {'A': 20, 'S': 8}


In [70]:
def classify(row, node):
    """See the 'rules of recursion' above."""
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if isinstance(node,Decision_Node):
        if node.question.match(row):
            return classify(row, node.true_branch)
        else:        
            return classify(row, node.false_branch)
        
    else:
        for i in range(len(node.all_branch)):
            if node.cutpoint[i] == row[node.question.column]:
                return classify(row,node.all_branch[i])
            

In [71]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.

classify(training_data[5], my_treeauc)

#######

{'A': 20, 'S': 8}

In [72]:
def print_leaf(counts,training_data):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    finalvalue=0
    for lbl in counts.keys():
        probs[lbl] = float(counts[lbl] / total )
    probs=list(probs.items())
    probs.sort()
    
    denomval=[0]*2
    
    for ls in training_data:
        if ls[-1]=='S':
            denomval[1]+=1
        else:
            denomval[0]+=1
    
    if len(probs)==1:
        if probs[0][0]=='A':
            return probs[0][0]
        else:
            return probs[1][0]
    
    #neg/(pos+neg)*P(+|x)>pos/(pos+neg)*P(-|x)
    if denomval[0]/len(training_data)*float(probs[1][1])>denomval[1]/len(training_data)*float(probs[0][1]):
        return probs[1][0]
    else:
        return probs[0][0]

In [73]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(training_data[54], my_treeauc),training_data)
#######

'S'

In [74]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(training_data[5], my_treeauc),training_data)
######

'S'

In [75]:
acc=0
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_treegini),training_data)))
    if row[-1] == print_leaf(classify(row, my_treegini),training_data):
        acc+=1
print(float(acc)/len(testing_data))

Actual: A. Predicted: S
Actual: A. Predicted: A
Actual: A. Predicted: A
Actual: S. Predicted: A
Actual: S. Predicted: A
Actual: S. Predicted: A
Actual: A. Predicted: A
Actual: A. Predicted: A
Actual: S. Predicted: S
Actual: A. Predicted: S
Actual: A. Predicted: S
Actual: A. Predicted: A
Actual: A. Predicted: S
Actual: A. Predicted: A
Actual: A. Predicted: A
Actual: A. Predicted: A
Actual: S. Predicted: A
Actual: A. Predicted: S
0.5


In [76]:
acc=0
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_treeratio),training_data)))
    if row[-1] == print_leaf(classify(row, my_treeratio),training_data):
        acc+=1
print(float(acc)/len(testing_data))

Actual: A. Predicted: S
Actual: A. Predicted: A
Actual: A. Predicted: A
Actual: S. Predicted: A
Actual: S. Predicted: A
Actual: S. Predicted: A
Actual: A. Predicted: A
Actual: A. Predicted: A
Actual: S. Predicted: S
Actual: A. Predicted: S
Actual: A. Predicted: S
Actual: A. Predicted: A
Actual: A. Predicted: S
Actual: A. Predicted: A
Actual: A. Predicted: A
Actual: A. Predicted: A
Actual: S. Predicted: A
Actual: A. Predicted: S
0.5


In [77]:
acc=0
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_treeauc),training_data)))
    if row[-1] == print_leaf(classify(row, my_treeauc),training_data):
        acc+=1
print(float(acc)/len(testing_data))

Actual: A. Predicted: S
Actual: A. Predicted: A
Actual: A. Predicted: A
Actual: S. Predicted: A
Actual: S. Predicted: A
Actual: S. Predicted: A
Actual: A. Predicted: S
Actual: A. Predicted: A
Actual: S. Predicted: A
Actual: A. Predicted: S
Actual: A. Predicted: A
Actual: A. Predicted: A
Actual: A. Predicted: S
Actual: A. Predicted: S
Actual: A. Predicted: A
Actual: A. Predicted: A
Actual: S. Predicted: A
Actual: A. Predicted: S
0.3888888888888889


In [78]:
def outcome(testing_data,my_tree,training_data):
    pos=0
    neg=0
    acc=0
    tp=0
    fp=0
    pre=0
    f1=0
    accuracy=None
    tprate=None
    fprate=None
    precision=None
    f1=None
    
    for row in testing_data:       
        if row[-1] =='S':
            pos+=1
        else:
            neg+=1
        if row[-1] == print_leaf(classify(row, my_tree),training_data):
            acc+=1
            if row[-1]=='S': 
                tp+=1
        else:
            if row[-1]=='A': 
                fp+=1
                
        accuracy=acc/len(testing_data)
        if pos!=0:
            tprate=tp/pos
        if neg!=0:
            fprate=fp/neg
        if tp!=0 or fp!=0:
            precision=tp/(tp+fp)
        if tprate!=None and precision!=None and tprate!=0 and precision!=0:
            f1=1/((1/tprate)+(1/precision))/2
            
    return print("Accuracy: %s. TP rate: %s. FP rate: %s. Precision: %s. F1: %s" % (accuracy,tprate,fprate,precision,f1))
        

In [86]:
#ten fold cross validation
random.seed(100)
randtran=random.sample(range(88),88)

for i in range(4):   
    testing_data=[data[k] for k in randtran[i*22:i*22+22]]
    training_data=[data[k] for k in range(88) if k not in randtran[i*22:i*22+22]]

    my_treegini=build_treegini(training_data,data)
    my_treeratio = build_treeratio(training_data,data)
    my_treeauc = build_treeauc(training_data,data) 
    outcome(testing_data,my_treegini,training_data)
    outcome(testing_data,my_treeratio,training_data)
    outcome(testing_data,my_treeauc,training_data)

Accuracy: 0.5909090909090909. TP rate: 0.0. FP rate: 0.13333333333333333. Precision: 0.0. F1: None
Accuracy: 0.5909090909090909. TP rate: 0.0. FP rate: 0.13333333333333333. Precision: 0.0. F1: None
Accuracy: 0.5909090909090909. TP rate: 0.0. FP rate: 0.13333333333333333. Precision: 0.0. F1: None
Accuracy: 0.45454545454545453. TP rate: 0.8. FP rate: 0.6470588235294118. Precision: 0.26666666666666666. F1: 0.1
Accuracy: 0.45454545454545453. TP rate: 0.8. FP rate: 0.6470588235294118. Precision: 0.26666666666666666. F1: 0.1
Accuracy: 0.36363636363636365. TP rate: 0.6. FP rate: 0.7058823529411765. Precision: 0.2. F1: 0.075
Accuracy: 0.36363636363636365. TP rate: 0.5. FP rate: 0.7142857142857143. Precision: 0.2857142857142857. F1: 0.09090909090909091
Accuracy: 0.36363636363636365. TP rate: 0.5. FP rate: 0.7142857142857143. Precision: 0.2857142857142857. F1: 0.09090909090909091
Accuracy: 0.3181818181818182. TP rate: 0.25. FP rate: 0.6428571428571429. Precision: 0.18181818181818182. F1: 0.05263